In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE # Pour avoir 50 % de 0 et 50 % de 1 dans la colonne presence_accident
from sklearn.linear_model import LogisticRegression # Regression logistique
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold #KFold répété, pour mieux entraîner le modèle
from sklearn.model_selection import train_test_split
import seaborn as sns #Tableau des corrélations
import matplotlib.pyplot as plt 
import plotly.express as px # Pour faire des graphes dans l'interface Dash

**Importation du dataframe df_utiles traité dans prep_df_utiles.ipynb**

In [2]:
df_utiles = pd.read_csv("df_utiles.csv")

In [15]:
df_utiles['lat'] = df_utiles['lat'].str.replace(',', '.').astype(float)
df_utiles['long'] = df_utiles['long'].str.replace(',', '.').astype(float)

# Pour avoir les coordonnées GPS en float, sinon plotly.express (px) ne peut pas les afficher sur la carte de France

In [4]:
df_utiles.columns # Liste des colonnes

Index(['Unnamed: 0', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'agg', 'int',
       'atm', 'col', 'lat', 'long', 'place', 'catu', 'grav', 'sexe', 'an_nais',
       'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp', 'catr',
       'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'surf', 'infra',
       'situ', 'vma', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'motor',
       'presence_deces'],
      dtype='object')

**Analyse statistique des données de df_utiles.csv : matrice des corrélations (en commentaire car fait planter Jupyter)**

In [5]:
#corr = df_utiles.drop(['an', 'presence_deces'], axis=1).corr()
#f, ax = plt.subplots(figsize=(11, 9))
#cmap = sns.diverging_palette(230, 20, as_cmap=True)
#sns.heatmap(corr, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

**Test ARIMA pour anticiper nbre accidents**

In [6]:
pip install statsmodels #statsmodels, pour utiliser le module de séries temp SARIMAX

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#statsmodels,'


In [7]:
# ARIMA
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.statespace.sarimax import SARIMAX # SARIMAX = modèle de séries temp

**But : obtenir la colonne nbre_acc_j qui représente le nombre d'accidents par jour**

In [8]:
from datetime import date
df_utiles = pd.read_csv('df_utiles.csv')
# la colonne "jour_format_normal" contient la date au format standard dd/mm/yyyy
df_utiles["jour_format_normal"] = 0 # Initialisation de la colonne
df_utiles["jour_format_normal"] = df_utiles["jour"].astype(str)+'/'+df_utiles["mois"].astype(str)+'/'+df_utiles["an"].astype(str)
df_utiles["jour_format_normal"] = pd.to_datetime(df_utiles["jour_format_normal"], format="%d/%m/%Y")
df_utiles["colonne_soustraction"] = date(df_utiles["an"].iloc[0], 1, 1) # Création d'une colonne remplie avec la date du 01/01/2019
df_utiles["colonne_soustraction"] = pd.to_datetime(df_utiles["colonne_soustraction"]) #Pour transformer en objet date 
df_utiles["nbre_j"] = df_utiles["jour_format_normal"] - df_utiles["colonne_soustraction"] # Permet d'obtenir une colonne contenant, pour chaque accident,
# Le nombre de jours écoulés depuis le 01/01/2019

def datification(x):
    return x.days # Permet d'obtenir l'entier contenu par l'objet "date" de Python
df_utiles["nbre_j"] = df_utiles["nbre_j"].apply(datification) #Maintenant nbre_j est une colonne d'entiers
nbre_acc_j = df_utiles.groupby(['nbre_j']).agg('count').jour.values #Nbre d'accidents (ou de victimes d'accidents jsp) par jour
ts = pd.Series(nbre_acc_j, index=pd.date_range("1/1/2019", periods=365)) # Objet Series de pandas, pour faire des graphes

**SARIMAX : prédire nbre d'accidents en s'appuyant sur les données déjà enregistrées**

**Graphe d'autocorrélation (les deux dernières lignes de code font planter Jupyter donc sont en commentaire)**

In [9]:
from statsmodels.graphics.tsaplots import plot_acf

# Subtract the rolling mean
nbre_acc_j_dataframe = pd.DataFrame(nbre_acc_j)
nbre_acc_rolling = nbre_acc_j_dataframe - nbre_acc_j_dataframe.rolling(15).mean()

# Drop the NaN values
nbre_acc_rolling = nbre_acc_rolling.dropna()

# Create figure and subplots
#fig, ax1 = plt.subplots()

# Plot the ACF
#plot_acf(nbre_acc_rolling, lags=25, zero=False, ax=ax1);

Conclusion : la composante saisonale a pour période 7 (abscisse du max sur le plot ci-dessus)

In [10]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Create a SARIMAX model
model = SARIMAX(nbre_acc_j, order=(5, 0, 0), seasonal_order=(1, 1, 0, 7)) # Le 7 correspond à la période déterminée ci-dessus

# Fit the model
results = model.fit()

# Print the results summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                                 y   No. Observations:                  365
Model:             SARIMAX(5, 0, 0)x(1, 1, 0, 7)   Log Likelihood               -1939.442
Date:                           Wed, 22 Dec 2021   AIC                           3892.884
Time:                                   11:01:33   BIC                           3920.048
Sample:                                        0   HQIC                          3903.687
                                           - 365                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2613      0.052      5.074      0.000       0.160       0.362
ar.L2          0.1273      0.057      2.221      0.026       0.015       0.240
ar.L3          0.0943      0.055      1.725      0.085      -0.013       0.201
ar.L4          0.0820      0.058      1.416      0.157      -0.031       0.195
ar.L5          0.0144      0.056      0.256      0.798      -0.096       0.125
ar.S.L7       -0.3782      0.051     -7.450      0.000      -0.478      -0.279
sigma2      2961.9202    210.258     14.087      0.000    2549.822    3374.019
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 4.15
Prob(Q):                              0.98   Prob(JB):                         0.13
Heteroskedasticity (H):               1.49   Skew:                            -0.13
Prob(H) (two-sided):                  0.03   Kurtosis:                         3.46
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [11]:
results_pred = results.get_forecast(31) #Utilise le modèle précédent afin de prédire 31 nouvelles valeurs (mois de janvier 2020)
results_mean = results_pred.predicted_mean #Je ne sais pas trop
results_mean = pd.DataFrame(results_mean) #En objet dataframe
results_final = nbre_acc_j_dataframe.append(results_mean) #On adjoint les 31 prédictions aux valeurs réelles afin de tout afficher
results_final = results_final.reset_index() #Après cette fusion de dataframe, l'index est cassé, on le réinitialise donc
results_final = results_final.drop("index", axis=1) #La commande précédent ajoute une colonne "index" inutile dans le DataFrame, on la supprime
date_index = pd.date_range('1/1/2019', periods=396, freq='D') #Pour avoir les dates en abscisse (365 + 31 = 396 jours)
results_final = results_final.set_index(date_index)
print(results_final)

                     0
2019-01-01  263.000000
2019-01-02  239.000000
2019-01-03  255.000000
2019-01-04  298.000000
2019-01-05  287.000000
...                ...
2020-01-27  288.703401
2020-01-28  285.480236
2020-01-29  209.320611
2020-01-30  274.991159
2020-01-31  317.720738

[396 rows x 1 columns]


# Récupération des frontières des régions françaises

In [12]:
fr_regions = pd.read_json('https://france-geojson.gregoiredavid.fr/repo/regions.geojson')
fr_regions = fr_regions.to_json()
print(df_utiles)

        Unnamed: 0  jour  mois    an  hrmn  lum  dep  agg  int  atm  ...  \
0                0    30    11  2019    60    4   93    1    1    1  ...   
1                1    30    11  2019    60    4   93    1    1    1  ...   
2                2    30    11  2019    60    4   93    1    1    1  ...   
3                3    30    11  2019   120    3   93    1    1    1  ...   
4                4    28    11  2019   900    1   92    1    1    1  ...   
...            ...   ...   ...   ...   ...  ...  ...  ...  ...  ...  ...   
132972      132972    27    11  2019   420    1   67    1    1    8  ...   
132973      132973    30    11  2019   120    4   94    1    1    1  ...   
132974      132974    30    11  2019   900    1   78    1    1    1  ...   
132975      132975    29    11  2019  1200    3   92    1    1    1  ...   
132976      132976    29    11  2019  1200    3   92    1    1    1  ...   

        catv obs obsm  choc  manv  motor  presence_deces  jour_format_normal  \
0      

In [13]:
variables_utiles_regr = ['nbre_j', 'hrmn', 'lum', 'lat', 'long', 'agg', 'int', 'atm', 'col', 'catr', 'vosp', 'prof', 'plan', 'surf', 'infra', 'situ', 'catv', 'obs', 'obsm', 'choc', 'manv', 'catu', 'sexe', 'presence_deces']
variables_utiles = ['nbre_j', 'hrmn', 'lum', 'lat', 'long', 'agg', 'col', 'prof', 'vma', 'catu', 'sexe']
variables_utiles_utiles = ['nbre_j', 'hrmn', 'lum', 'lat', 'long', 'agg', 'col', 'prof', 'vma', 'catu', 'sexe', 'presence_deces']

def regression_logistique(var_choisies, pred):
    df_rayon = []
    df_rayon = df_utiles[variables_utiles_utiles]
    df_rayon = pd.DataFrame(df_rayon)
    X = df_rayon[variables_utiles]
    y = df_rayon.presence_deces #colonne "presence_deces"
    #print(y)
    y = y.astype(int)
    os = SMOTE(random_state=0) #Initialisation de SMOTE, pour avoir 50 % de 0 et 50 % de 1 dans y (sans modifier la distribution de l'échantillon d'accidents)
    cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1) #K-Fold répété, augmente la précision de l'algo de ~10 %
    for train_index, test_index in cv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train,y_train=os.fit_resample(X_train, y_train) #On applique OS initialisé plus tôt
    #X_train = pd.DataFrame(data=X_train)
    #y_train= pd.DataFrame(data=y_train)
    logreg = LogisticRegression(solver='lbfgs') # Initialisation de la régression logistique
    logreg.fit(X_train, y_train) # Fitting sur l'ensemble de train
    y_pred = logreg.predict(X_test) # Prédictions
    #print(y_pred)
    #print(logreg.coef_)
    #print('Précision de la classification par régression logistique : ',logreg.score(X_test, y_test)*100,"%")
    pred_pred = logreg.predict(pred)
    precision_pred = max(logreg.predict_proba(pred)[0][0], logreg.predict_proba(pred)[0][1])*100
    return [pred_pred, precision_pred, logreg.score(X_test, y_test)*100]

In [16]:
## Dash app (https://dash.plot.ly/getting-started)

import numpy as np

import dash # Dash
import dash_core_components as dcc # Pour utiliser dcc.Input (ce qui permet d'inclure les inputs dans le HTML)
import dash_html_components as html # L'interface Dash est en HTML
from dash.dependencies import Input, Output, State, MATCH, ALL # Pour les callbacks

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] # Feuille CSS externe, pour mettre en forme la dashboard

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

ALLOWED_VAR = (
    'nbre_j', 'hrmn', 'lum', 'lat', 'long', 'agg', 'col', 'prof', 'vma', 'catu', 'sexe'
) # Liste des variables sur lesquelles on utilise l'algo

app.layout = html.Div(children=[ #Toute l'app est une balise div, qui contient des balises H1, ..., H6 (pour les titres) et des balises Div pour tous les contenus interactifs
    html.H1(children='Prédictions'),
    html.Div([
        dcc.Graph(id="Carte",figure=px.scatter_geo(df_utiles.head(1000), lat="lat", lon="long", geojson=fr_regions))] # Carte affichant les accidents sur une carte de France
    ,style={'columnCount': 1}), 
    html.Div([
        dcc.Graph(id="Prédictions",figure=px.line(results_final, labels={'x': 'jour', 'y': 'nombre d\'accidentés'}))] #Prédictions SARIMAX
    ,style={'columnCount': 1}), 
    html.Div(children='''
        Remplir les variables
    '''),
            html.Div( # Cette balise Div permet d'afficher les inputs sur l'interface Dash, pour qu'on puisse les remplir
    [
        dcc.Input(
            id="input_{}".format(_), # On identifie chaque Input par un ID "input_nomdelavariable", utile pour les callbacks
            type="number",
            placeholder="variable {}".format(_),
        )
        for _ in ALLOWED_VAR
    ]
    + [html.Div(id="out-all-types")]
)
])

@app.callback(
     Output("out-all-types", "children"),
     [Input("input_{}".format(_), "value") for _ in ALLOWED_VAR], # On lie les inputs affichés sur Dash et la fonction cb_render
)

def cb_render(*vals):
    if not None in vals:
        float_vals = []
        for item in vals:
            float_vals.append(float(item))
        res = regression_logistique(variables_utiles, (np.reshape(vals, (1, -1))))
        return "L'algorithme prédit : {} avec une probabilité de {} et une précision de : {}".format(res[0], res[1], res[2]) 

if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port='8050')

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Dec/2021 11:04:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:04:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:04:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:04:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 11:05:08] "POST /_dash-update-component

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\thomas\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\thomas\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\thomas\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\thomas\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\thomas\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\thomas\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\thomas\Anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    respons

127.0.0.1 - - [22/Dec/2021 11:05:19] "POST /_dash-update-component HTTP/1.1" 500 -
